# Trading Pipeline Demo

Demonstrates running the pipeline for a date range and a single day.

In [ ]:
import sys, os
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)


In [ ]:
from trading_bot.agents import MarketAnalystAgent, RiskAdvisorAgent, NewsSummarizerAgent
from trading_bot.coordinator import Coordinator
from trading_bot.pipeline import Pipeline
from trading_bot.storage import JSONStorage
from trading_bot.backtest import Backtester
from trading_bot.portfolio import Portfolio

coord = Coordinator([MarketAnalystAgent(), RiskAdvisorAgent(), NewsSummarizerAgent()])
storage = JSONStorage()
backtester = Backtester()
portfolio = Portfolio()
pipe = Pipeline(coord, storage=storage, backtester=backtester, portfolio=portfolio)


In [ ]:
range_result = pipe.run('TSLA', start_date='2025-01-01', end_date='2025-01-03')
range_result['files'], range_result['backtest']


In [ ]:
single_result = pipe.run('TSLA', today='2025-01-02')
single_result['strategy'], single_result['files']


In [ ]:
portfolio.total_pnl(), [p.__dict__ for p in portfolio.closed_positions()]
